In [1]:
import pandas as pd 
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.dialects.sqlite import DATETIME

In [2]:
df = pd.read_csv('../Resources/collision_geo_data_clean.csv')

In [3]:
df.set_index('collision_id', inplace=True)

In [4]:
df['zip_code'] = df['zip_code'].astype(str)

In [5]:
df['crash_datetime'] = df['crash_date'] + ' ' + df['crash_time']

In [6]:
df.drop(columns=['crash_date', 'crash_time'], inplace=True)

In [7]:
df['crash_datetime'] = pd.to_datetime(df['crash_datetime'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 4455765 to 4407712
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   street_name                    1000 non-null   object        
 1   borough                        1000 non-null   object        
 2   county                         1000 non-null   object        
 3   zip_code                       1000 non-null   object        
 4   latitude                       1000 non-null   float64       
 5   longitude                      1000 non-null   float64       
 6   number_of_persons_injured      1000 non-null   int64         
 7   number_of_persons_killed       1000 non-null   int64         
 8   number_of_pedestrians_injured  1000 non-null   int64         
 9   number_of_pedestrians_killed   1000 non-null   int64         
 10  number_of_cyclist_injured      1000 non-null   int64         
 11  number_o

In [9]:
import requests
import pandas as pd

# Fetch data from the API
additional_columns = requests.get("https://data.cityofnewyork.us/resource/h9gi-nx95.json")
additional_columns_data = additional_columns.json()
additional_df = pd.DataFrame(additional_columns_data)
additional_df.set_index('collision_id', inplace=True)
additional_df.head(5)

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,latitude,longitude,location,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
collision_id,,,,,,,,,,,,,,,,,,,,,
4455765,2021-09-11T00:00:00.000,2:39,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4513547,2022-03-26T00:00:00.000,11:45,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4541903,2022-06-29T00:00:00.000,6:55,THROGS NECK BRIDGE,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4456314,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,40.667202,-73.8665,"{'latitude': '40.667202', 'longitude': '-73.86...",1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4486609,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,40.683304,-73.917274,"{'latitude': '40.683304', 'longitude': '-73.91...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
additional_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4455765 to 4407712
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     1000 non-null   object
 1   crash_time                     1000 non-null   object
 2   on_street_name                 742 non-null    object
 3   off_street_name                451 non-null    object
 4   number_of_persons_injured      1000 non-null   object
 5   number_of_persons_killed       1000 non-null   object
 6   number_of_pedestrians_injured  1000 non-null   object
 7   number_of_pedestrians_killed   1000 non-null   object
 8   number_of_cyclist_injured      1000 non-null   object
 9   number_of_cyclist_killed       1000 non-null   object
 10  number_of_motorist_injured     1000 non-null   object
 11  number_of_motorist_killed      1000 non-null   object
 12  contributing_factor_vehicle_1  997 non-null    object
 13 

In [11]:
df.index = df.index.astype(str)
additional_df.index = additional_df.index.astype(str)

In [12]:
# rename columns contributing_factor_vehicle to contributing_factor_vehicle_1 
df.rename(columns={'contributing_factor_vehicle':'contributing_factor_vehicle_1'}, inplace=True)

In [13]:
df['contributing_factor_vehicle_2'] = additional_df['contributing_factor_vehicle_2']
df['contributing_factor_vehicle_3'] = additional_df['contributing_factor_vehicle_3']
df['contributing_factor_vehicle_4'] = additional_df['contributing_factor_vehicle_4']
df['contributing_factor_vehicle_5'] = additional_df['contributing_factor_vehicle_5']

In [14]:
df['vehicle_type_code_1'] = additional_df['vehicle_type_code1']
df['vehicle_type_code_2'] = additional_df['vehicle_type_code2']
df['vehicle_type_code_3'] = additional_df['vehicle_type_code_3']
df['vehicle_type_code_4'] = additional_df['vehicle_type_code_4']
df['vehicle_type_code_5'] = additional_df['vehicle_type_code_5']

In [15]:
crash_datetime = df['crash_datetime']
df.drop(labels=['crash_datetime'], axis=1, inplace=True)
df.insert(0, 'crash_datetime', crash_datetime)

In [16]:
import datetime as dt
# convert the datetime to days of week 
position = df.columns.get_loc('crash_datetime') + 1
crash_day_of_week = df['crash_datetime'].dt.dayofweek
df.insert(position, 'crash_day_of_week', crash_day_of_week)
df.head(5)

,crash_datetime,crash_day_of_week,street_name,borough,county,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,...,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
collision_id,,,,,,,,,,,,,,,,,,,,,
4455765,2021-09-11 02:39:00,5,Whitestone Expressway & 20 Avenue,Queens,Queens County,11356,40.712728,-74.006015,2,0,...,Aggressive Driving/Road Rage,Unspecified,NaN,NaN,NaN,Sedan,Sedan,NaN,NaN,NaN
4513547,2022-03-26 11:45:00,5,Queensboro Bridge Upper,Manhattan,Queens County,10044,40.746402,-73.940191,1,0,...,Pavement Slippery,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN
4541903,2022-06-29 06:55:00,2,Throgs Neck Bridge,Bronx,Bronx County,10465,40.816416,-73.798616,0,0,...,Following Too Closely,Unspecified,NaN,NaN,NaN,Sedan,Pick-up Truck,NaN,NaN,NaN
4456314,2021-09-11 09:35:00,5,1211 Loring Avenue,Brooklyn,Kings County,11208,40.667202,-73.866500,0,0,...,Unspecified,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN
4486609,2021-12-14 08:13:00,1,Saratoga Avenue & Decatur Street,Brooklyn,Kings County,11233,40.683304,-73.917274,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df.loc['4277087', 'street_name'] = 'Bruckner Blvd & E 138 Street'

In [18]:
demo_df = pd.read_excel('../Resources/NewYork_DemographicsByZipCode.xlsx')
demo_df = demo_df.iloc[3:]
demo_df = demo_df.rename(columns={'Geography': 'zip_code'}, inplace=False)
demo_df = demo_df.rename(columns={'Best Population Estimate': 'Population'}, inplace=False)
demo_df = demo_df.rename(columns={demo_df.columns[7]: 'Population Density'}, inplace=False)
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 3 to 1830
Data columns (total 67 columns):
 #   Column                                                                       Non-Null Count  Dtype 
---  ------                                                                       --------------  ----- 
 0   zip_code                                                                     1828 non-null   object
 1   GEOID                                                                        1828 non-null   object
 2   State                                                                        1828 non-null   object
 3   County                                                                       6 non-null      object
 4   City                                                                         6 non-null      object
 5   Best Population Year                                                         1828 non-null   object
 6   Population                                      

In [19]:
demo_df = demo_df[['zip_code','Population Density','Population']]
demo_df.head()

,zip_code,Population Density,Population
3,name,population_density_sq_mi,population
4,New York,417.6,19677151
5,06390,14.3,58
6,10001,43227.1,26966
7,10002,93406.1,76807


In [20]:
df = df.merge(demo_df, how='left')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 1000 non-null   datetime64[ns]
 1   crash_day_of_week              1000 non-null   int64         
 2   street_name                    1000 non-null   object        
 3   borough                        1000 non-null   object        
 4   county                         1000 non-null   object        
 5   zip_code                       1000 non-null   object        
 6   latitude                       1000 non-null   float64       
 7   longitude                      1000 non-null   float64       
 8   number_of_persons_injured      1000 non-null   int64         
 9   number_of_persons_killed       1000 non-null   int64         
 10  number_of_pedestrians_injured  1000 non-null   int64         
 11  number_of_pedestri

In [22]:
collision_by_zip_df = df.groupby('zip_code').size()
collision_by_zip_df = collision_by_zip_df.reset_index().rename(columns={"zip_code":"zipcodes",0:"# of Collisions"})
collision_by_zip_df['# of Collisions'] = collision_by_zip_df['# of Collisions'].astype('int')
collision_by_zip_df['zipcodes'] = collision_by_zip_df['zipcodes'].astype('str')
collision_by_zip_df.head()

,zipcodes,# of Collisions
0,10001,4
1,10002,9
2,10003,3
3,10007,11
4,10009,3


In [23]:
engine = create_engine('sqlite:///../Resources/collision_db.sqlite')

In [24]:
metadata = MetaData()

In [25]:
collision_by_zip = Table('collision_by_zip', metadata,
    Column('zipcodes', String, primary_key=True),
    Column('# of Collisions', Integer),
)

In [26]:
collision_by_zip_df.to_sql('collision_by_zip', engine, if_exists='replace', index=True)

170

In [27]:
motor_collisions = Table('motor_collisions', metadata,
                         Column('collision_id', Integer, primary_key=True),
                         Column('crash_datetime', DateTime),
                         Column('crash_day_of_week', Integer),
                         Column('street_name', String), 
                         Column('borough', String),
                         Column('county', String),
                         Column('zip_code', String),
                         Column('latitude', Float),
                         Column('longitude', Float),
                         Column('number_of_persons_injured', Integer),
                         Column('number_of_persons_killed', Integer),
                         Column('number_of_pedestrians_injured', Integer),
                         Column('number_of_pedestrians_killed', Integer),
                         Column('number_of_cyclist_injured', Integer),
                         Column('number_of_cyclist_killed', Integer),
                         Column('number_of_motorist_injured', Integer),
                         Column('number_of_motorist_killed', Integer),
                         Column('contributing_factor_vehicle_1', String),
                         Column('contributing_factor_vehicle_2', String),
                         Column('contributing_factor_vehicle_3', String),
                         Column('contributing_factor_vehicle_4', String),
                         Column('contributing_factor_vehicle_5', String),
                         Column('vehicle_type_code_1', String),
                         Column('vehicle_type_code_2', String),
                         Column('vehicle_type_code_3', String),
                         Column('vehicle_type_code_4', String),
                         Column('vehicle_type_code_5', String),
                         Column('Population', String),
                         Column('Population Density', String)
                         )
metadata.create_all(engine)

In [28]:
df.to_sql('motor_collisions', con=engine, if_exists='replace', index=True)

1000

In [29]:
from sqlalchemy import inspect

inspector = inspect(engine)
print(inspector.get_table_names())  # Check if the table is listed
print(inspector.get_columns('motor_collisions'))  # Check the structure of the table


['collision_by_zip', 'motor_collisions']
[{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'crash_datetime', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'crash_day_of_week', 'type': BIGINT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'street_name', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'borough', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'county', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'zip_code', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'latitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'longitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'number_of_persons_injured', 'type': BIGINT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name